# Extraction of Laws References from the quiz questions

In [30]:
import pandas as pd
import json
import os
import re

isLinux = True
default_linux_path = os.getcwd().replace("/Data", "/Documents/Downloaded") if "/Data" in os.getcwd() else os.getcwd() + "/Documents/Downloaded"
default_windows_path = os.getcwd().replace("\\Data", "\\Documents\\Downloaded") if "\\Data" in os.getcwd() else os.getcwd() + "\\Documents\\Downloaded"
default_path = default_linux_path if isLinux else default_windows_path

DEFAULT_SAVE_DIR = default_path.replace("/Downloaded", "/Generated") if isLinux else default_path.replace("\\Downloaded", "\\Generated")
LAWS_CSV = DEFAULT_SAVE_DIR + ('/laws.csv' if isLinux else '\\laws.csv')
REFERENCES_CSV = DEFAULT_SAVE_DIR + '/references_merged.csv'
QUIZZES_CSV = DEFAULT_SAVE_DIR + '/quiz_merged.csv'

df_quiz = pd.read_csv(DEFAULT_SAVE_DIR + '/quiz_merged.csv')
df_laws = pd.read_csv(LAWS_CSV)

In [30]:
def retrieve_law(source, law_id):
    if source == 'c.p.':
        result = df_laws.loc[(df_laws['Source'] == "c.p.") & (df_laws['Article'] == f"Art. {law_id}."), 'Comma content']
    elif source == 'c.p.p.':
        result = df_laws.loc[(df_laws['Source'] == "c.p.p.") & (df_laws['Article'] == f"Art. {law_id}."), 'Comma content']
    else:
        return None

    if not result.empty:
        return result.iloc[0]
    return None
    

def extract_data(df_quiz, regex_patterns, source):
    extracted_data = []
    no_match_data = []

    for i, row in df_quiz.iterrows():
        question = row['Question']
        match = None

        for pattern in regex_patterns:
            match = re.findall(pattern, question)
            question_plh = re.sub(pattern, '{PLH}', question)
            if match:
                break

        if match:
            # Search for any comma reference
            comma = re.findall(r'comma ([^ ^\.^,]+)', question)
            if comma:
                comma = comma[0].strip()
            else:
                comma = None
            
            # Catch of set of laws not single
            if source == 'D. Lgs.':
                for i, elem in enumerate(match):
                    if type(elem) is tuple:
                        reference = elem[0]
                        groupSource = elem[1].replace(' del ', '/')
                    elif type(elem) is str:
                        reference = None
                        groupSource = elem.replace(' del ', '/')
                    else:
                        raise Exception(f'Unexpected type {type(elem)}')
                    
                    extracted_data.append({
                        'Source': groupSource.strip(),
                        'Comma': comma,
                        'Reference': reference.strip() if reference else None,
                        'Question id': row['Index'],
                        'Question plh': question_plh[i],
                        'Law text': retrieve_law(groupSource, reference.strip() if reference else None)
                    })
            else:
                if type(match[0]) is tuple:
                    print(source)
                reference = match[0]
                extracted_data.append({
                    'Source': source,
                    'Comma': comma,
                    'Reference': reference[0].strip() if type(reference) is tuple else reference.strip(),
                    'Question id': row['Index'],
                    'Question plh': question_plh,
                    'Law text': retrieve_law(source, reference[0].strip() if type(reference) is tuple else reference.strip())
                })
        else:
            no_match_data.append(row.to_dict())

    df_cp = pd.DataFrame(extracted_data)
    df_no_match = pd.DataFrame(no_match_data)

    return df_cp, df_no_match

## Extraction of Penal Code's References

In [31]:
patterns = [
    r'[Aa]rtt?\.( \d+).{0,25}c\.p\.[^p^a]',
    r"(?:articolo|art\.)( ?\d+) del [Cc]odice [Pp]enale",
    r'[Cc]odice penale all\'art. (\d+)'
]

df_cp, df_quiz = extract_data(df_quiz, patterns, "c.p.")

print("CP found: ", df_cp.shape)
print("Still unmatched: ", df_quiz.shape)
df_cp.head()

CP found:  (153, 6)
Still unmatched:  (967, 5)


,Source,Comma,Reference,Question id,Question plh,Law text
0,c.p.,None,240,224,Ai sensi dell'{PLH}Ã¨ sempre ordinata la confi...,"(Confisca) Nel caso di condanna, il giudice ..."
1,c.p.,None,266,225,"Ai sensi dell'{PLH}Ã¨ punito, se il fatto non ...",(Istigazione di militari a disobbedire alle le...
2,c.p.,None,24,226,La pena della multa ex {PLH}consiste nel pagam...,(Multa) La pena della multa consiste nel pag...
3,c.p.,None,7,227,Ai sensi dell'{PLH}Ã¨ punito secondo la legge ...,(Reati commessi all'estero) È punito secondo...
4,c.p.,None,19,228,Tra le pene accessorie per i delitti ex {PLH}t...,None


## Extraction of Administrative Penal Code's References

In [32]:
patterns = [
    r'[Aa]rtt?\.( \d+).{0,30}[Cc]\.p\.a[\.]?',
    r'(?:[Aa]rticolo|[Aa]rtt?\.)( \d+).{0,30} del [Cc]odice del [Pp]rocesso [Aa]mministrativo'
]

df_cpa, df_quiz_quiz = extract_data(df_quiz, patterns, "c.p.a.")

print("CPA found: ", df_cpa.shape)
print("Still unmatched: ", df_quiz.shape)
df_cpa.head()

CPA found:  (58, 6)
Still unmatched:  (967, 5)


,Source,Comma,Reference,Question id,Question plh,Law text
0,c.p.a.,None,80,2,Ai sensi dell'{PLH} come avviene la prosecuzio...,None
1,c.p.a.,None,42,3,Entro quale termine le parti devono proporre r...,None
2,c.p.a.,None,133,4,"A norma di quanto dispone l'{PLH}, salvo ulter...",None
3,c.p.a.,None,7,9,Ai sensi dell'{PLH} la giurisdizione amministr...,None
4,c.p.a.,None,34,10,Le pronunce definitive del giudice possono ess...,None


## Extraction of Procedural Penal Code's References

In [33]:
patterns = [
    r'[Aa]rtt?\.( \d+).{0,25}[Cc]\.p\.p[\.]?',
    r'articolo( \d+).{0,25}[Cc]\.p\.p[\.]?',
    r'(?:[Aa]rticolo|[Aa]rtt?\.)( \d+).{0,30} del [Cc]odice di [Pp]rocedura [Pp]enale'
]

df_cpp, df_quiz = extract_data(df_quiz, patterns, "c.p.p.")

print("CPP found: ", df_cpp.shape)
print("Still unmatched: ", df_quiz.shape)
df_cpp.head()

CPP found:  (205, 6)
Still unmatched:  (762, 5)


,Source,Comma,Reference,Question id,Question plh,Law text
0,c.p.p.,None,57,270,Quali tra questi NON sono agenti di polizia gi...,None
1,c.p.p.,None,57,271,Quali tra questi NON sono ufficiali di polizia...,None
2,c.p.p.,None,59,272,L'ufficiale preposto ai servizi di polizia giu...,None
3,c.p.p.,None,71,273,Se ex {PLH} risulta che lo stato mentale dell'...,None
4,c.p.p.,None,71,274,Con l'ordinanza di sospensione del procediment...,None


## Extraction of Costitution's References

In [34]:
patterns = [
    r'[Aa]rtt?\.?( \d+).{0,25}[Cc]ost'
]

df_cost, df_quiz = extract_data(df_quiz, patterns, "constitution")

print("Costitution found: ", df_cost.shape)
print("Still unmatched: ", df_quiz.shape)
df_cost.head()

Costitution found:  (30, 6)
Still unmatched:  (732, 5)


,Source,Comma,Reference,Question id,Question plh,Law text
0,constitution,None,117,5,Consacrando a livello costituzionale i princip...,None
1,constitution,None,25,461,L'{PLH}ituzione disponendo che nessuno puÃ² es...,None
2,constitution,None,117,616,Consacrando a livello costituzionale i princip...,None
3,constitution,None,118,619,Indicare quale principio costituz ionale della...,None
4,constitution,None,126,631,Qualora con decreto motivato e nei casi previs...,None


## Extraction of Legislative Decree's References (NEED TO CATCH ALSO THE REFERENCE ARTICLE)

In [35]:
patterns = [
    r'decreto legislativo n\.( ?\d+ del \d+)',
    r'[Aa]rtt?\.?( \d+).{0,25}D\.Lgs\.(?: n\.)?( \d+\/\d+)',
    r'[Aa]rtt?\.?( \d+).{0,25}D\.Lgs\. n\.( \d+ del \d+)',
    r'D\.Lgs\.(?: n\.)?( \d+\/ ?\d+)',
]

df_dlgs, df_quiz = extract_data(df_quiz, patterns, "D. Lgs.")

print("Laws found: ", df_dlgs.shape)
print("Still unmatched: ", df_quiz.shape)
df_dlgs.head()

Laws found:  (263, 6)
Still unmatched:  (479, 5)


,Source,Comma,Reference,Question id,Question plh,Law text
0,33/2013,None,5,6,L,None
1,50/2016,None,23,14,A,None
2,165/2001,None,34,24,C,None
3,165/2001,None,33,26,D,None
4,165/2001,1,16,28,I,None


## Extraction of Laws References

In [36]:
patterns = [
    r'[Aa]rtt?\.?( \d+).{0,25}[lL]\. ?n\.( \d+\/\d+)',
    r'[Aa]rtt?\.?( \d+).{0,25}[lL]\.( \d+\/\d+)',
    r'n\.( \d+\/\d+)',
    r'[Ll]egge( \d+\/\d+)',
    r'l\.( \d+\/\d+)',
]

def custom_match(match):
    article = match.group(1)
    comma = match.group(2)
    return article, comma

df_laws, df_quiz = extract_data(df_quiz, patterns, "Legge")

print("Laws found: ", df_laws.shape)
print("Still unmatched: ", df_quiz.shape)
df_laws.head()

Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Legge
Laws found:  (265, 6)
Still unmatched:  (214, 5)


,Source,Comma,Reference,Question id,Question plh,Law text
0,Legge,None,241/1990,1,Il Capo II della l. {PLH} è riservato alla reg...,None
1,Legge,None,241/1990,7,"Nel procedimento amministrativo, cosa prevede ...",None
2,Legge,None,9,8,"A norma del disposto di cui all'{PLH}, chi può...",None
3,Legge,None,16,11,"L'{PLH} quale termine prevede, dal ricevimento...",None
4,Legge,None,3,12,Per quanto riguarda il contenuto della motivaz...,None


## Check Missing Rows

In [37]:
# Print Question column of elements that were not matched
for i, row in df_quiz.iterrows():
    print(row['Question'])

Ai sensi dell'art. 80 C.p.a. come avviene la prosecuzione del giudizio in caso di sospensione? 
Entro quale termine le parti devono proporre ricorso incidentale nell'ambito del processo amministrativo ai sensi dell'art. 42 C.p.a.? 
A norma di quanto dispone l'art. 133 del C.p.a., salvo ulteriori previsioni di legge, a chi sono devolute le controversie aventi ad oggetto i provvedimenti relativi alla disciplina o al divieto dell'esercizio d'industrie insalubri o pericolose? 
Ai sensi dell'art. 7 C.p.a. la giurisdizione amministrativa si articola in giurisdizione generale di legittimità, esclusiva ed estesa al merito. Cosa si intende per quella di legittimità? 
Le pronunce definitive del giudice possono essere di merito (art. 34 c.p.a.) o di rito (art. 35 c.p.a.). Quale tra le seguenti è una pronuncia di merito? 
Nell'espletamento delle procedure semplificate di cui all'art. 36, le stazioni appaltanti garantiscono l'effettiva contendibilità degli affidamenti da parte dei soggetti potenzia

## Export the data

In [39]:
# Merge the dataframes and clean up the data
df_merged = pd.concat([df_cp, df_cpa, df_cpp, df_cost, df_dlgs, df_laws], ignore_index=True)
df_merged.to_csv(DEFAULT_SAVE_DIR + '/references_merged.csv', index=False)

print(df_merged.shape)
print(df_merged.head())

# Generate the quizzes csv with plhs
df_merged = df_merged.dropna(subset=['Question plh'])
df_merged = df_merged.drop_duplicates(subset=['Question plh'])
df_quiz = pd.read_csv(DEFAULT_SAVE_DIR + '/quiz_merged.csv')
df_quiz_plh = pd.merge(df_merged, df_quiz, left_on='Question id', right_on='Index', how='inner')
df_quiz_plh = df_quiz_plh.drop(columns=['Source', 'Comma', 'Reference','Question', 'Index'])
df_quiz_plh.rename(columns={'Question id': 'Index'}, inplace=True)

print(df_quiz_plh.shape)
print(df_quiz_plh.head())

df_quiz_plh = df_quiz_plh.sort_values(by='Index')
df_quiz_plh.to_csv(DEFAULT_SAVE_DIR + '/quiz_merged_plh.csv', index=False)

(974, 6)
  Source Comma Reference  Question id  \
0   c.p.  None       240          224   
1   c.p.  None       266          225   
2   c.p.  None        24          226   
3   c.p.  None         7          227   
4   c.p.  None        19          228   

                                        Question plh  \
0  Ai sensi dell'{PLH}Ã¨ sempre ordinata la confi...   
1  Ai sensi dell'{PLH}Ã¨ punito, se il fatto non ...   
2  La pena della multa ex {PLH}consiste nel pagam...   
3  Ai sensi dell'{PLH}Ã¨ punito secondo la legge ...   
4  Tra le pene accessorie per i delitti ex {PLH}t...   

                                            Law text  
0  (Confisca)   Nel caso di condanna, il giudice ...  
1  (Istigazione di militari a disobbedire alle le...  
2  (Multa)   La pena della multa consiste nel pag...  
3  (Reati commessi all'estero)   È punito secondo...  
4                                               None  
(726, 6)
   Index                                       Question plh  \
0    

## Extraction of a Queries-Pos-Neg dataset

In [34]:
df_quiz = pd.read_csv(QUIZZES_CSV)
df_references = pd.read_csv(REFERENCES_CSV)
df_laws = pd.read_csv(LAWS_CSV)
dataset = [] # query, pos, neg

print(df_quiz.columns)#'quiz_id', 'question', 'answer_1', 'answer_2', 'answer_3'
print(df_references.columns)#Source', 'Comma', 'Reference', 'Question id', 'Question plh','Law text
print(df_laws.columns)#'Source', 'Article', 'Comma number', 'Comma content'

# print Source and Comma fields where Law text is nan
print(df_references.loc[df_references['Law text'].isna() & df_references["Source"] != "Legge", ['Source', 'Reference']])

for i, row in df_references.iterrows():
    query = df_quiz.loc[df_quiz['quiz_id'] == row['Question id'], 'question'].iloc[0]    
    pos = row['Law text']
    
    if pd.isna(pos):
        continue
    
    for _ in range(3):
        neg = df_laws.sample(1)['Comma content'].iloc[0]
        dataset.append({
            'query': query,
            'pos': pos,
            'neg': neg
        })
print(len(dataset))
print(dataset[-1])
print(dataset[-2])
print(dataset[-3])

with open(DEFAULT_SAVE_DIR + '/training_data.json', 'w+') as f:
    json.dump(dataset, f)

Index(['quiz_id', 'question', 'answer_1', 'answer_2', 'answer_3'], dtype='object')
Index(['Source', 'Comma', 'Reference', 'Question id', 'Question plh',
       'Law text'],
      dtype='object')
Index(['Source', 'Article', 'Comma number', 'Comma content'], dtype='object')
    Source Reference
0     c.p.       240
1     c.p.       266
2     c.p.        24
3     c.p.         7
4     c.p.        19
..     ...       ...
969  Legge  300/1999
970  Legge  112/1999
971  Legge         2
972  Legge         2
973  Legge         2

[974 rows x 2 columns]
399
{'query': 'Quale mezzo di impugnazione è ammissibile avverso l’ordinanza di sospensione necessaria del processo ex art. 295 c.p.c.?', 'pos': "(Attentato contro i Capi di Stati esteri)   Chiunque nel territorio dello Stato attenta alla vita, alla incolumità o alla libertà personale del Capo di uno Stato estero è punito, nel caso di attentato alla vita, con la reclusione non inferiore a venti anni e, negli altri casi, con la reclusione non infer